In [ ]:
## Analysing closely best clustering results

In [2]:
import pandas as pd
df_1 = pd.read_csv('coded_transcript_5k_nofalcon.csv')
df_2 = pd.read_csv('coded_transcript_5k_falcon.csv')
merged_df = pd.merge(df_1, df_2[['utterance', 'Falcon_Code']], on='utterance', how='left')
df_3 = pd.read_csv('coded_transcript_10k.csv')
df_4 = pd.read_csv('coded_transcript_20k.csv')
combined_df = pd.concat([merged_df, df_3, df_4], ignore_index=True)
mixtral_df_1 = pd.read_csv('debatesrag_mixtral_1k_NL.csv')
mixtral_df_2 = pd.read_csv('debatesrag_mixtral_2k_NL.csv')
mixtral_df_4 = pd.read_csv('debatesrag_mixtral_4k_NL.csv')
mixtral_df_5 = pd.read_csv('debatesrag_mixtral_5k_NL.csv')
mixtral_df_7 = pd.read_csv('debatesrag_mixtral_7k_NL.csv')
mixtral_df_8 = pd.read_csv('debatesrag_mixtral_8k_NL.csv')
mixtral_df_9 = pd.read_csv('debatesrag_mixtral_9k_NL.csv')
mixtral_df_10 = pd.read_csv('debatesrag_mixtral_10k_NL.csv')
mixtral_df_12 = pd.read_csv('debatesrag_mixtral_12k_NL.csv')
mixtral_df_13 = pd.read_csv('debatesrag_mixtral_13k_NL.csv')
mixtral_df_14 = pd.read_csv('debatesrag_mixtral_14k_NL.csv')
mixtral_df_16 = pd.read_csv('debatesrag_mixtral_16k_NL.csv')
mixtral_df_17 = pd.read_csv('debatesrag_mixtral_17k_NL.csv')
mixtral_df_18 = pd.read_csv('debatesrag_mixtral_18k_NL.csv')
mixtral_df_20 = pd.read_csv('debatesrag_mixtral_20k_NL.csv')
extra_df = combined_df[:20000]
mixtral_df = pd.concat([mixtral_df_1, mixtral_df_2, mixtral_df_4,mixtral_df_5,mixtral_df_7,mixtral_df_8,
                       mixtral_df_9, mixtral_df_10, mixtral_df_12,mixtral_df_13,mixtral_df_14,
                       mixtral_df_16,mixtral_df_17, mixtral_df_18, mixtral_df_20], ignore_index=True)


In [17]:
assert len(mixtral_df) == len(extra_df), "Row count mismatch between dataframes!"

# Add desired columns from extra_df to mixtral_df
mixtral_df['speaker'] = extra_df['speaker']
mixtral_df['party'] = extra_df['party']
mixtral_df['source_doc_title'] = extra_df['source_doc_title']
mixtral_df['topic'] = extra_df['topic']
mixtral_df.to_csv('mixtral_df.csv')

In [18]:
def clean_after_linebreak(text):
    if pd.isna(text):
        return text
    return text.split('\n')[0]  # keep only part before the first line break

mixtral_df['Final_Code'] = mixtral_df['Final_Code'].apply(clean_after_linebreak)
import pandas as pd

def clean_generated_code(text):
    if pd.isna(text):
        return text
    # Keep only before line break
    text = text.split('\n')[0]
    # Remove explanation phrases if they exist
    explanation_phrases = [
        "This code captures",
        "This code encapsulates",
        "This sentence captures",
        "This code reflects",
        "This phrase conveys",
        "which means",
        '''" or ''',
        '''" ''',
        "meaning that",
        "which reflects",
        "indicating that"
    ]
    for phrase in explanation_phrases:
        if phrase in text:
            text = text.split(phrase)[0].strip()
    return text.strip(' "')

# Apply to DataFrame
mixtral_df['Final_Code'] = mixtral_df['Final_Code'].apply(clean_generated_code)
mixtral_df["Model"] = "Mixtral"

def get_top_final_codes(df, model_name, top_n=20):
    code_counts = df["Final_Code"].value_counts().head(top_n).reset_index()
    code_counts.columns = ["Final_Code", "Frequency"]
    code_counts["Model"] = model_name
    return code_counts

top_mixtral_codes = get_top_final_codes(mixtral_df, "Mixtral")

top_codes_df = pd.concat([top_mixtral_codes], ignore_index=True)
top_codes_df
mixtral_df.to_csv('mixtral_df.csv')


In [28]:

mixtral_df[["topic_1", "topic_2"]] = mixtral_df["topic"].str.split(" \| ", n=1, expand=True)

# How many unique categories?
n_topic1 = mixtral_df["topic_1"].nunique()
n_topic2 = mixtral_df["topic_2"].nunique()

print(f"Unique first-level topics (topic_1): {n_topic1}")
print(f"Unique second-level topics (topic_2): {n_topic2}")



Unique first-level topics (topic_1): 16
Unique second-level topics (topic_2): 28


In [9]:
results_df = pd.read_csv('clustering_sweep_results.csv')

In [11]:
top10 = results_df.sort_values("score_for_rank", ascending=False).head(10)

In [12]:
top10

,embedding,reduction,algo,k,params,silhouette,davies_bouldin,calinski_harabasz,n_clusters,noise,noise_rate,silhouette_core,davies_bouldin_core,calinski_harabasz_core,n_labeled,score_for_rank
38,all-MiniLM-L6-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 20}",-0.021197,1.056955,65.597389,18.0,19115.0,0.95575,0.746790,0.543215,1349.377930,885.0,0.746790
197,all-mpnet-base-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 20}",-0.049798,1.097393,65.834679,17.0,19107.0,0.95535,0.731481,0.550296,1375.397217,893.0,0.731481
37,all-MiniLM-L6-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 10}",-0.055091,1.065092,40.163879,41.0,18774.0,0.93870,0.706806,0.565662,722.329346,1226.0,0.706806
356,multi-qa-MiniLM-L6-cos-v1,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 20}",-0.041875,1.113653,64.516815,17.0,19045.0,0.95225,0.698999,0.632631,1429.068115,955.0,0.698999
195,all-mpnet-base-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 5}",-0.101009,1.107312,18.789003,121.0,18222.0,0.91110,0.692333,0.550902,357.611542,1778.0,0.692333
196,all-mpnet-base-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 10}",-0.092507,1.129356,35.690186,44.0,18744.0,0.93720,0.692152,0.596360,690.431152,1256.0,0.692152
36,all-MiniLM-L6-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 5}",-0.066236,1.035713,21.821249,108.0,18288.0,0.91440,0.681731,0.554299,379.015503,1712.0,0.681731
513,paraphrase-MiniLM-L3-v2,none,DBSCAN,NaN,"{'eps': 0.3, 'min_samples': 5}",-0.079331,1.004000,23.667665,94.0,18372.0,0.91860,0.667805,0.495045,430.770477,1628.0,0.667805
632,paraphrase-MiniLM-L3-v2,umap,HDBSCAN,NaN,{'min_cluster_size': 10},-0.038494,0.785907,277.959686,302.0,10080.0,0.50400,0.667673,0.421343,15385.746094,9920.0,0.667673
156,all-MiniLM-L6-v2,umap,HDBSCAN,NaN,{'min_cluster_size': 20},-0.076658,1.026014,264.858673,151.0,10128.0,0.50640,0.666552,0.453693,13327.109375,9872.0,0.666552


In [14]:
coded1 = pd.read_csv('coded_transcript_20k.csv')
coded2 = pd.read_csv('coded_transcript_20k.csv')

In [16]:
coded['topic']

0       Bestuur | Parlement
1       Bestuur | Parlement
2       Bestuur | Parlement
3       Bestuur | Parlement
4       Bestuur | Parlement
               ...         
9995    Bestuur | Parlement
9996    Bestuur | Parlement
9997    Bestuur | Parlement
9998    Bestuur | Parlement
9999    Bestuur | Parlement
Name: topic, Length: 10000, dtype: object

In [31]:
api_key = ''

In [28]:
mixtral_df

,utterance,Llama3_Code,Falcon_Code,Mistral_Code,Generated_Code,Final_Code,speaker,party,source_doc_title,Model,topic,topic_1,topic_2
0,Ik stel voor de beraadslaging over deze wetson...,This is a transcript of a meeting of the Dutch...,NaN,The main idea of this sentence is that the spe...,"""Dutch Parliamentary Debate""",Dutch Parliamentary Debate,De voorzitter,NaN,Handelingen Eerste Kamer 1907-1908 08 februari...,Mixtral,NaN,NaN,NaN
1,"Ik geef thans het woord aan den heer Regout, d...","The main idea of this sentence is: ""The speake...",<noinput> - <noinput><|endoftext|>,This sentence discusses the exchange of a priv...,"""Speaker's explanation of private letter excha...",Speaker's explanation of private letter exchange,De voorzitter,NaN,Handelingen Eerste Kamer 1907-1908 08 februari...,Mixtral,NaN,NaN,NaN
2,Aan de orde is de eerste termijn van de behand...,"Main idea: ""Discussion of a bill to update ani...",The main idea of the sentence is that the firs...,This sentence is about welcoming the Minister ...,"""Welcoming Minister for animal health bill dis...",Welcoming Minister for animal health bill disc...,De voorzitter,NaN,"Vergadering nr. 32, item 9 (2024) - Actualiser...",Mixtral,Landbouw | Dieren,Landbouw,Dieren
3,Ik geef het woord aan de heer Jaspers namens d...,Main idea: Jaspers will give a maiden speech o...,The main idea of the sentence is that the spea...,"This sentence means: ""I give the word to Lord ...",Jaspers' maiden speech by BBB representative,Jaspers' maiden speech by BBB representative,De voorzitter,NaN,"Vergadering nr. 32, item 9 (2024) - Actualiser...",Mixtral,Landbouw | Dieren,Landbouw,Dieren
4,Voorzitter. Laat ik beginnen met iets wat we v...,This is a speech by a member of the Dutch parl...,The main idea of the sentence is that the spea...,The main idea of this speech is advocating for...,"""Advocating Humane Livestock Practices""",Advocating Humane Livestock Practices,Jaspers,BBB,"Vergadering nr. 32, item 9 (2024) - Actualiser...",Mixtral,Landbouw | Dieren,Landbouw,Dieren
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,"Van harte steun, voorzitter. Ik wil namelijk j...",Support.<|eot_id|><|start_header_id|>assistant...,The main idea of the sentence is that the spea...,"The chairman has your support, but you wish to...",Support for Foreign Affairs Minister.,Support for Foreign Affairs Minister.,De voorzitter,NaN,"Vergadering nr. 74, item 33 (2024) - Het niet ...",Mixtral,Zorg en gezondheid | Organisatie en beleid,Zorg en gezondheid,Organisatie en beleid
19996,Steun.,Support.<|eot_id|><|start_header_id|>assistant...,Please provide a summary of the sentence.,"The word ""Steun"" is Dutch for ""support"" or ""help",Dutch: Support / Help,"Dutch ""Steun"": Support or Help",Krul,CDA,"Vergadering nr. 74, item 33 (2024) - Het niet ...",Mixtral,Zorg en gezondheid | Organisatie en beleid,Zorg en gezondheid,Organisatie en beleid
19997,Steun.,Support.<|eot_id|><|start_header_id|>assistant...,Please provide a summary of the sentence.,"The word ""Steun"" is Dutch for ""support"" or ""help","Dutch ""Steun"": Support or Help",Dutch: Support/Help,De voorzitter,NaN,"Vergadering nr. 74, item 33 (2024) - Het niet ...",Mixtral,Zorg en gezondheid | Organisatie en beleid,Zorg en gezondheid,Organisatie en beleid
19998,"U heeft geen meerderheid, meneer Van Baarle.","No majority, Mr. Van Baarle. (Main idea: No",Van Baarle has no advantage over you.<|endofte...,"Mr. Van Baarle, U does not have a majority.</s>","""Mr. Van Baarle lacks majority""",Mr. Van Baarle lacks majority,Bushoff,GroenLinks-PvdA,"Vergadering nr. 74, item 33 (2024) - Het niet ...",Mixtral,Zorg en gezondheid | Organisatie en beleid,Zorg en gezondheid,Organisatie en beleid


In [34]:
# --- installs (run once in your env; skip if you already have them) ---
# %pip install -q sentence-transformers umap-learn hdbscan scikit-learn

import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
import umap
import hdbscan

mixtral_df["text"] = (mixtral_df["Final_Code"].fillna("").astype(str) + " " +
                      mixtral_df["utterance"].fillna("").astype(str))
text_data = mixtral_df["text"].tolist()

# ---------------------------
# CONFIG pulled from screenshot
# ---------------------------
METHODS = [
    # (column_name, embedding_model_id, reduction, algo, kwargs)
    ("dbscan_minilm_none_ms20",
     "sentence-transformers/all-MiniLM-L6-v2", "none", "DBSCAN", {"eps": 0.3, "min_samples": 20}),
    ("dbscan_mpnet_none_ms20",
     "sentence-transformers/all-mpnet-base-v2", "none", "DBSCAN", {"eps": 0.3, "min_samples": 20}),
    ("dbscan_minilm_none_ms10",
     "sentence-transformers/all-MiniLM-L6-v2", "none", "DBSCAN", {"eps": 0.3, "min_samples": 10}),
    ("hdbscan_paraphrase_umap_mcs10",
     "sentence-transformers/paraphrase-MiniLM-L3-v2", "umap", "HDBSCAN", {"min_cluster_size": 10}),
    ("hdbscan_minilm_umap_mcs20",
     "sentence-transformers/all-MiniLM-L6-v2", "umap", "HDBSCAN", {"min_cluster_size": 20}),
    ("hdbscan_bge_umap_mcs50",
     "BAAI/bge-base-en-v1.5", "umap", "HDBSCAN", {"min_cluster_size": 50}),
    ("kmeans_e5_umap_k4",
     "intfloat/e5-small-v2", "umap", "KMeans", {"n_clusters": 4, "random_state": 42, "n_init": "auto"}),
    ("gmm_e5_umap_c4",
     "intfloat/e5-small-v2", "umap", "GMM", {"n_components": 4, "random_state": 42}),
    ("spectral_mpnet_umap_k4_nn",
     "sentence-transformers/all-mpnet-base-v2", "umap", "Spectral",
     {"n_clusters": 4, "affinity": "nearest_neighbors", "assign_labels": "kmeans", "random_state": 42}),
    ("agglo_e5_umap_k8",
     "intfloat/e5-small-v2", "umap", "Agglomerative", {"n_clusters": 8, "linkage": "ward"}),
]

# UMAP settings (not in screenshot; pick a sensible default)
UMAP_KW = dict(n_components=10, n_neighbors=15, min_dist=0.0, random_state=42, metric="cosine")

# ---------------------------
# Helpers with caching
# ---------------------------
_embed_cache = {}
_reduce_cache = {}

def get_embeddings(model_id: str, texts: list[str]) -> np.ndarray:
    if model_id not in _embed_cache:
        model = SentenceTransformer(model_id)
        _embed_cache[model_id] = np.asarray(model.encode(texts, show_progress_bar=True, convert_to_numpy=True))
    return _embed_cache[model_id]

def reduce_features(key: str, X: np.ndarray, method: str) -> np.ndarray:
    """
    key ties to (model_id + method) so we reuse reductions when multiple algos share same inputs.
    """
    if method == "none":
        return X
    cache_key = (key, UMAP_KW["n_components"], UMAP_KW["n_neighbors"], UMAP_KW["min_dist"], UMAP_KW["metric"])
    if cache_key not in _reduce_cache:
        reducer = umap.UMAP(**UMAP_KW)
        _reduce_cache[cache_key] = reducer.fit_transform(X)
    return _reduce_cache[cache_key]

def run_clustering(algo: str, features: np.ndarray, kwargs: dict) -> np.ndarray:
    if algo == "DBSCAN":
        labels = DBSCAN(**kwargs, metric="cosine").fit_predict(features)
    elif algo == "HDBSCAN":
        # HDBSCAN uses distance by default; give metric='euclidean' if UMAP produced euclidean-embedded space.
        # If using raw embeddings, cosine often works better:
        labels = hdbscan.HDBSCAN(metric="euclidean", **kwargs).fit_predict(features)
    elif algo == "KMeans":
        labels = KMeans(**kwargs).fit_predict(features)
    elif algo == "GMM":
        gm = GaussianMixture(**kwargs)
        gm.fit(features)
        labels = gm.predict(features)
    elif algo == "Spectral":
        labels = SpectralClustering(**kwargs).fit_predict(features)
    elif algo == "Agglomerative":
        labels = AgglomerativeClustering(**kwargs).fit_predict(features)
    else:
        raise ValueError(f"Unknown algo: {algo}")
    return labels

# ---------------------------
# Execute all 10 methods
# ---------------------------
texts = text_data  # from your prep
for col_name, model_id, reduction, algo, kwargs in METHODS:
    # 1) embeddings
    X = get_embeddings(model_id, texts)

    # 2) optional reduction
    Xr = reduce_features(f"{model_id}_{reduction}", X, reduction)

    # 3) clustering
    labels = run_clustering(algo, Xr, kwargs)

    # 4) write column to dataframe
    mixtral_df[col_name] = labels.astype(int)

# Done. Inspect the columns now present:
new_cols = [m[0] for m in METHODS]
mixtral_df[new_cols].head()


Batches: 100%|██████████| 625/625 [00:07<00:00, 85.58it/s] 
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.

,dbscan_minilm_none_ms20,dbscan_mpnet_none_ms20,dbscan_minilm_none_ms10,hdbscan_paraphrase_umap_mcs10,hdbscan_minilm_umap_mcs20,hdbscan_bge_umap_mcs50,kmeans_e5_umap_k4,gmm_e5_umap_c4,spectral_mpnet_umap_k4_nn,agglo_e5_umap_k8
0,0,0,0,-1,-1,-1,0,0,0,3
1,0,0,0,-1,-1,58,0,0,0,3
2,0,0,0,289,107,-1,2,2,0,5
3,0,0,0,179,95,-1,1,2,0,0
4,0,0,0,-1,107,29,0,0,0,3


In [35]:
cluster_counts = {}

for col in [
    "dbscan_minilm_none_ms20",
    "dbscan_mpnet_none_ms20",
    "dbscan_minilm_none_ms10",
    "hdbscan_paraphrase_umap_mcs10",
    "hdbscan_minilm_umap_mcs20",
    "hdbscan_bge_umap_mcs50",
    "kmeans_e5_umap_k4",
    "gmm_e5_umap_c4",
    "spectral_mpnet_umap_k4_nn",
    "agglo_e5_umap_k8"
]:
    labels = mixtral_df[col]
    # exclude -1 (noise) if present
    n_clusters = len(set(labels)) - (1 if -1 in labels.values else 0)
    cluster_counts[col] = n_clusters

# Show nicely as a DataFrame
import pandas as pd
cluster_counts_df = pd.DataFrame.from_dict(cluster_counts, orient="index", columns=["n_clusters"])
cluster_counts_df


,n_clusters
dbscan_minilm_none_ms20,4
dbscan_mpnet_none_ms20,3
dbscan_minilm_none_ms10,8
hdbscan_paraphrase_umap_mcs10,330
hdbscan_minilm_umap_mcs20,171
hdbscan_bge_umap_mcs50,60
kmeans_e5_umap_k4,4
gmm_e5_umap_c4,4
spectral_mpnet_umap_k4_nn,4
agglo_e5_umap_k8,8


In [32]:
# ====== 0) Imports ======
import re, time, numpy as np, pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Dict, Any

# Embeddings
from sentence_transformers import SentenceTransformer

# Reductions
from sklearn.decomposition import PCA
import umap

# Clustering
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.mixture import GaussianMixture
import hdbscan

# External metrics
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score, adjusted_mutual_info_score,
    homogeneity_completeness_v_measure
)

# ====== 1) Prepare text ======
assert {"Final_Code","utterance","topic"}.issubset(mixtral_df.columns), "mixtral_df must have Final_Code, utterance, topic"
mixtral_df["text"] = (mixtral_df["Final_Code"].fillna("") + " " +
                      mixtral_df["utterance"].fillna("")).str.strip()

# Split taxonomy into two levels
def split_topics(df, topic_col="topic"):
    if "topic_1" not in df.columns or "topic_2" not in df.columns:
        df[["topic_1","topic_2"]] = df[topic_col].astype(str).str.split(r"\s\|\s", n=1, expand=True)
        df["topic_2"] = df["topic_2"].fillna(df["topic_1"])  # if only one level present
    return df

mixtral_df = split_topics(mixtral_df)

# ====== 2) Configs from your Table 4 ======
# You can tweak k / min_cluster_size etc. if needed
@dataclass
class Config:
    name: str
    embedding: str
    reduction: Optional[str]          # "none" | "umap" | "pca"
    clustering: str                   # "DBSCAN"|"HDBSCAN"|"KMeans"|"Agglo"|"Spectral"|"GMM"
    params: Dict[str, Any]

top10 = [
    Config("dbscan_allMiniLM_none",          "all-MiniLM-L6-v2", "none",  "DBSCAN",   dict(eps=1.0, min_samples=5)),
    Config("hdbscan_paraphraseL3_umap",      "paraphrase-MiniLM-L3-v2", "umap", "HDBSCAN", dict(min_cluster_size=15, min_samples=5)),
    Config("kmeans_e5_umap_k4",              "intfloat/e5-small", "umap", "KMeans",   dict(n_clusters=4, random_state=42)),
    Config("agglo_e5_umap_k8",               "intfloat/e5-small", "umap", "Agglo",    dict(n_clusters=8, linkage="ward")),
    Config("gmm_e5_umap_k4",                 "intfloat/e5-small", "umap", "GMM",      dict(n_components=4, random_state=42)),
    Config("spectral_mpnet_umap_k4",         "all-mpnet-base-v2", "umap", "Spectral", dict(n_clusters=4, random_state=42, affinity="nearest_neighbors", n_neighbors=10)),
    Config("kmeans_bgebase_pca_k5",          "BAAI/bge-base-en",  "pca",  "KMeans",   dict(n_clusters=5, random_state=42)),
    Config("hdbscan_multiqa_umap",           "multi-qa-MiniLM-L6-cos-v1", "umap", "HDBSCAN", dict(min_cluster_size=20, min_samples=5)),
    Config("agglo_e5base_umap_k6",           "intfloat/e5-base",  "umap", "Agglo",    dict(n_clusters=6, linkage="ward")),
    Config("kmeans_bgelarge_none_k7",        "BAAI/bge-large-en", "none", "KMeans",   dict(n_clusters=7, random_state=42)),
]

# ====== 3) Embedding cache so we don’t recompute ======
_model_cache: Dict[str, SentenceTransformer] = {}
_embed_cache: Dict[str, np.ndarray] = {}

def get_embeddings(model_name: str, texts: list[str]) -> np.ndarray:
    if model_name not in _embed_cache:
        if model_name not in _model_cache:
            _model_cache[model_name] = SentenceTransformer(model_name)
        _embed_cache[model_name] = _model_cache[model_name].encode(texts, show_progress_bar=True, normalize_embeddings=True)
    return _embed_cache[model_name]

# ====== 4) Reduction helpers ======
def apply_reduction(X: np.ndarray, reduction: Optional[str]) -> np.ndarray:
    if reduction is None or reduction == "none":
        return X
    if reduction == "pca":
        return PCA(n_components=min(50, X.shape[1]), random_state=42).fit_transform(X)
    if reduction == "umap":
        return umap.UMAP(n_components=15, random_state=42).fit_transform(X)
    raise ValueError(reduction)

# ====== 5) Clustering runner ======
def run_one_config(df: pd.DataFrame, cfg: Config, text_col="text") -> np.ndarray:
    X = get_embeddings(cfg.embedding, df[text_col].tolist())
    Xr = apply_reduction(X, cfg.reduction)

    cl = cfg.clustering.lower()
    p  = cfg.params

    if cl == "kmeans":
        labels = KMeans(**p).fit_predict(Xr)
    elif cl == "hdbscan":
        labels = hdbscan.HDBSCAN(**p).fit_predict(Xr)
    elif cl == "dbscan":
        labels = DBSCAN(**p).fit_predict(Xr)
    elif cl == "agglo":
        labels = AgglomerativeClustering(**p).fit_predict(Xr)
    elif cl == "spectral":
        labels = SpectralClustering(**p).fit_predict(Xr)
    elif cl == "gmm":
        labels = GaussianMixture(**p).fit(Xr).predict(Xr)
    else:
        raise ValueError(cfg.clustering)
    return labels

# ====== 6) Execute all configs and add columns ======
for cfg in top10:
    col = cfg.name
    if col in mixtral_df.columns:
        continue
    print(f"Running {cfg.name} …")
    labels = run_one_config(mixtral_df, cfg)
    mixtral_df[col] = labels

print("Done. New clustering columns created:")
print([c for c in mixtral_df.columns if any(k in c for k in ["dbscan","hdbscan","kmeans","agglo","spectral","gmm"])])

# ====== 7) (Optional) Autoname clusters with Groq ======
# -- you can skip this block; evaluation will fall back to TF-IDF names.
from groq import Groq

def autoname_clusters_with_groq(df, cluster_col, api_key, model="llama-3.3-70b-versatile",
                                examples_per_cluster=5, code_col="Final_Code", utt_col="utterance",
                                max_retries=3, sleep_sec=1.0):
    client = Groq(api_key=api_key)
    names = {}
    work = df[df[cluster_col] != -1] if (df[cluster_col] == -1).any() else df
    for cid, g in work.groupby(cluster_col, sort=True):
        ex = g[[code_col, utt_col]].dropna().head(examples_per_cluster)
        if ex.empty:
            names[cid] = "Unlabeled cluster"; continue
        body = "\n\n".join([f'Code: "{str(r[code_col])[:180]}"\nUtterance: "{str(r[utt_col])[:300]}"'
                            for _, r in ex.iterrows()])
        prompt = f"""
You are a political discourse analyst performing axial coding.
Here are {len(ex)} coded examples from the same cluster:

{body}

Return ONE short axial code (≤5 words). Output ONLY the label.
"""
        lab = None
        for t in range(max_retries):
            try:
                resp = client.chat.completions.create(
                    model=model,
                    messages=[{"role":"system","content":"You are an expert in qualitative research and political discourse analysis."},
                              {"role":"user","content":prompt}],
                    temperature=0.2, max_tokens=16
                )
                lab = (resp.choices[0].message.content or "").strip().strip('"')
                break
            except Exception:
                time.sleep(sleep_sec)
        names[cid] = lab or "Unlabeled cluster"
    return names

# Optional: actually call Groq (set your key)
USE_GROQ = False
GROQ_API_KEY = api_key
llm_names = {}
if USE_GROQ:
    for cfg in top10:
        print("Autonaming:", cfg.name)
        llm_names[cfg.name] = autoname_clusters_with_groq(mixtral_df, cfg.name, api_key=GROQ_API_KEY)

# ====== 8) TF-IDF fallback naming (used when USE_GROQ=False) & evaluation ======
from sklearn.feature_extraction.text import TfidfVectorizer

_word = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9]+")
def normalize(s: str) -> str:
    return " ".join(_word.findall(str(s).lower()))

def tfidf_name(texts, topk=4):
    vec = TfidfVectorizer(ngram_range=(1,2), max_features=5000, min_df=2)
    X = vec.fit_transform([normalize(t) for t in texts if isinstance(t, str)])
    if X.shape[0] == 0: return "cluster"
    scores = np.asarray(X.mean(axis=0)).ravel()
    feats = np.array(vec.get_feature_names_out())
    idx = np.argsort(-scores)[:topk]
    return " ".join(feats[idx])

def build_cluster_names(df, cluster_col, provided=None, text_cols=("Final_Code","utterance")):
    if provided: return dict(provided)
    out = {}
    for cid, g in df.groupby(cluster_col):
        texts = []
        for c in text_cols:
            if c in g.columns:
                texts += g[c].dropna().astype(str).tolist()
        out[cid] = tfidf_name(texts[:1000])
    return out

# ROUGE-L (short labels)
def lcs_length(x, y):
    m, n = len(x), len(y)
    dp = [0]*(n+1)
    for i in range(1, m+1):
        prev = 0
        for j in range(1, n+1):
            tmp = dp[j]
            dp[j] = prev + 1 if x[i-1]==y[j-1] else max(dp[j], dp[j-1])
            prev = tmp
    return dp[n]
def rouge_l_f1(h, r):
    hx, ry = normalize(h).split(), normalize(r).split()
    if not hx or not ry: return 0.0
    lcs = lcs_length(hx, ry)
    if lcs==0: return 0.0
    p, rc = lcs/len(hx), lcs/len(ry)
    return 0.0 if p+rc==0 else 2*p*rc/(p+rc)

def evaluate_against_taxonomy(df, cluster_col, name_map=None):
    tmp = df[df[cluster_col] != -1] if (df[cluster_col]==-1).any() else df
    clabels = tmp[cluster_col].values

    t1 = tmp["topic_1"].astype(str)
    t2 = tmp["topic_2"].astype(str)

    # majority vote mapping for structure metrics
    def to_int(s): 
        lut = {k:i for i,k in enumerate(s.unique())}
        return s.map(lut), lut
    t1_int,_ = to_int(t1); t2_int,_ = to_int(t2)

    def majority_map(c, gold):
        mp = {}
        for cid in np.unique(c):
            mask = (c==cid)
            if mask.sum()==0: continue
            mp[cid] = pd.Series(gold[mask]).mode().iloc[0]
        return mp

    map1 = majority_map(clabels, t1_int.values)
    map2 = majority_map(clabels, t2_int.values)
    pred1 = np.array([map1.get(c,-1) for c in clabels])
    pred2 = np.array([map2.get(c,-1) for c in clabels])

    ari1 = adjusted_rand_score(t1_int, pred1)
    ami1 = adjusted_mutual_info_score(t1_int, pred1)
    h1,c1,v1 = homogeneity_completeness_v_measure(t1_int, pred1)

    ari2 = adjusted_rand_score(t2_int, pred2)
    ami2 = adjusted_mutual_info_score(t2_int, pred2)
    h2,c2,v2 = homogeneity_completeness_v_measure(t2_int, pred2)

    # names
    names = build_cluster_names(tmp, cluster_col, provided=name_map)
    topic1_names = sorted(tmp["topic_1"].dropna().unique().tolist())
    topic2_names = sorted(tmp["topic_2"].dropna().unique().tolist())

    def avg_max_rouge(name_list):
        return float(np.mean([max([rouge_l_f1(n, t) for t in name_list] or [0]) for n in names.values()]))
    return {
        "cluster_col": cluster_col,
        "n_clusters": int(tmp[cluster_col].nunique()),
        "ARI_topic1": ari1, "AMI_topic1": ami1, "V_measure_topic1": v1,
        "ARI_topic2": ari2, "AMI_topic2": ami2, "V_measure_topic2": v2,
        "name_rouge_topic1": avg_max_rouge(topic1_names),
        "name_rouge_topic2": avg_max_rouge(topic2_names),
    }, names

# Evaluate all 10
rows, name_banks = [], {}
for cfg in top10:
    provided = llm_names.get(cfg.name) if USE_GROQ else None
    scores, names = evaluate_against_taxonomy(mixtral_df, cfg.name, name_map=provided)
    rows.append(scores); name_banks[cfg.name] = names

eval_df = pd.DataFrame(rows).sort_values("V_measure_topic1", ascending=False)
print(eval_df)


Running dbscan_allMiniLM_none …


Batches: 100%|██████████| 625/625 [00:11<00:00, 52.43it/s] 


Running hdbscan_paraphraseL3_umap …


Batches: 100%|██████████| 625/625 [00:06<00:00, 89.70it/s] 
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Running kmeans_e5_umap_k4 …


...

Running agglo_e5base_umap_k6 …


Batches: 100%|██████████| 625/625 [00:54<00:00, 11.44it/s]
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Running kmeans_bgelarge_none_k7 …


Batches: 100%|██████████| 625/625 [02:48<00:00,  3.70it/s]


Done. New clustering columns created:
['dbscan_allMiniLM_none', 'hdbscan_paraphraseL3_umap', 'kmeans_e5_umap_k4', 'agglo_e5_umap_k8', 'gmm_e5_umap_k4', 'spectral_mpnet_umap_k4', 'kmeans_bgebase_pca_k5', 'hdbscan_multiqa_umap', 'agglo_e5base_umap_k6', 'kmeans_bgelarge_none_k7']
                 cluster_col  n_clusters  ARI_topic1  AMI_topic1  \
1  hdbscan_paraphraseL3_umap         253    0.086261    0.083637   
7       hdbscan_multiqa_umap         205    0.064176    0.079869   
9    kmeans_bgelarge_none_k7           7    0.002278    0.009976   
3           agglo_e5_umap_k8           8    0.001027    0.004965   
4             gmm_e5_umap_k4           4    0.000868    0.003666   
2          kmeans_e5_umap_k4           4    0.000868    0.003666   
8       agglo_e5base_umap_k6           6    0.000819    0.002944   
5     spectral_mpnet_umap_k4           4    0.000039    0.000310   
0      dbscan_allMiniLM_none           1    0.000000    0.000000   
6      kmeans_bgebase_pca_k5           5  

In [45]:
# # Or evaluate many columns and pass the per-column maps:
# top10_cols = [
#     "dbscan_allMiniLM",
#     "hdbscan_paraphraseL3_umap",
#     "kmeans_e5_umap_k4",
#     "agglo_e5_umap_k8",
#     "gmm_e5_umap_k4",
#     "spectral_mpnet_umap_k4",
#     "kmeans_bgebase_pca_k5",
#     "hdbscan_multiqa_umap",
#     "agglo_e5_umap_k6",
#     "kmeans_bgelarge_k7",
# ]

# # Build LLM labels for each column you want (do this in a loop if needed)
# llm_names = {}
# for col in top10_cols:
#     llm_names[col] = autoname_clusters_with_groq(
#         mixtral_df, cluster_col=col, api_key="YOUR_GROQ_API_KEY",
#         model="llama-3.3-70b-versatile", examples_per_cluster=5
#     )

# results_df, name_banks = evaluate_many(mixtral_df, top10_cols, llm_names=llm_names)
# print(results_df.sort_values("V_measure_topic1", ascending=False).head(10))

# Test set

In [3]:
import pandas as pd
extra_df = pd.read_csv('coded_transcript_test.csv')
mixtral_df = pd.read_csv('debatesrag_mixtral_test_NL.csv')

In [4]:
mixtral_df['speaker'] = extra_df['speaker']
mixtral_df['party'] = extra_df['party']
mixtral_df['source_doc_title'] = extra_df['source_doc_title']

In [5]:
def clean_after_linebreak(text):
    if pd.isna(text):
        return text
    return text.split('\n')[0]  # keep only part before the first line break

mixtral_df['Final_Code'] = mixtral_df['Final_Code'].apply(clean_after_linebreak)

def clean_generated_code(text):
    if pd.isna(text):
        return text
    # Keep only before line break
    text = text.split('\n')[0]
    # Remove explanation phrases if they exist
    explanation_phrases = [
        "This code captures",
        "This code encapsulates",
        "This sentence captures",
        "This code reflects",
        "This phrase conveys",
        "which means",
        '''" or ''',
        '''" ''',
        "meaning that",
        "which reflects",
        "indicating that"
    ]
    for phrase in explanation_phrases:
        if phrase in text:
            text = text.split(phrase)[0].strip()
    return text.strip(' "')

# Apply to DataFrame
mixtral_df['Final_Code'] = mixtral_df['Final_Code'].apply(clean_generated_code)


In [6]:
mixtral_df["Model"] = "Mixtral"

def get_top_final_codes(df, model_name, top_n=20):
    code_counts = df["Final_Code"].value_counts().head(top_n).reset_index()
    code_counts.columns = ["Final_Code", "Frequency"]
    code_counts["Model"] = model_name
    return code_counts

top_mixtral_codes = get_top_final_codes(mixtral_df, "Mixtral")

top_codes_df = pd.concat([top_mixtral_codes], ignore_index=True)
top_codes_df



,Final_Code,Frequency,Model
0,Amendment rejection vote outcome,251,Mixtral
1,Motion proposal by Van der Lee,166,Mixtral
2,Concerns over elderly care budget cuts,115,Mixtral
3,"Minister queried on funds allocation, includin...",45,Mixtral
4,Amendment rejection outcome,38,Mixtral
5,Minister's budget allocations discussion,38,Mixtral
6,Dutch: Support / Help,32,Mixtral
7,Delegate details to new Culture Minister,32,Mixtral
8,No support,31,Mixtral
9,Expresses support,30,Mixtral


In [7]:
import os
import json
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score, adjusted_mutual_info_score,
    normalized_mutual_info_score, v_measure_score, fowlkes_mallows_score
)
from scipy.optimize import linear_sum_assignment
import umap
import hdbscan
from sentence_transformers import SentenceTransformer

# ==== 1) Load test set ====
mixtral_df = pd.read_csv("debatesrag_mixtral_test_NL.csv")

# Build text input
mixtral_df["text"] = (
    mixtral_df["Final_Code"].fillna("").astype(str) + " " +
    mixtral_df["utterance"].fillna("").astype(str)
)
text_data = mixtral_df["text"].tolist()

# ==== 2) Embeddings + reductions ====
def reduce_none(X): return X
def reduce_pca(X): return PCA(n_components=min(64, X.shape[1]), random_state=42).fit_transform(X)
def reduce_umap(X): return umap.UMAP(n_components=15, random_state=42).fit_transform(X)

reducers = {"none": reduce_none, "pca": reduce_pca, "umap": reduce_umap}

model_map = {
    "all-MiniLM-L6-v2": "all-MiniLM-L6-v2",
    "all-mpnet-base-v2": "all-mpnet-base-v2",
    "paraphrase-MiniLM-L3-v2": "paraphrase-MiniLM-L3-v2",
    "e5-small": "intfloat/e5-small",
    "bge-base-en": "BAAI/bge-base-en",
}

def embed_texts(name, texts):
    model = SentenceTransformer(model_map[name])
    return model.encode(texts, show_progress_bar=True, normalize_embeddings=True)

embedding_cache, reduced_cache = {}, {}
def get_reduced(embedding_name, reduction_name):
    key = (embedding_name, reduction_name)
    if key in reduced_cache: return reduced_cache[key]
    if embedding_name not in embedding_cache:
        embedding_cache[embedding_name] = embed_texts(embedding_name, text_data)
    E = embedding_cache[embedding_name]
    X = reducers[reduction_name](E)
    reduced_cache[key] = X
    return X

# ==== 3) Best configs from 20k sweep ====
configs = [
    dict(name="dbscan_minilm_none_ms20", embedding="all-MiniLM-L6-v2", reduction="none",
         algo="DBSCAN", params={"eps": 0.3, "min_samples": 20}),
    dict(name="dbscan_mpnet_none_ms20", embedding="all-mpnet-base-v2", reduction="none",
         algo="DBSCAN", params={"eps": 0.3, "min_samples": 20}),
    dict(name="dbscan_minilm_none_ms10", embedding="all-MiniLM-L6-v2", reduction="none",
         algo="DBSCAN", params={"eps": 0.3, "min_samples": 10}),
    dict(name="hdbscan_paraphrase_umap_mcs10", embedding="paraphrase-MiniLM-L3-v2", reduction="umap",
         algo="HDBSCAN", params={"min_cluster_size": 10}),
    dict(name="hdbscan_minilm_umap_mcs20", embedding="all-MiniLM-L6-v2", reduction="umap",
         algo="HDBSCAN", params={"min_cluster_size": 20}),
    dict(name="hdbscan_bge_umap_mcs50", embedding="bge-base-en", reduction="umap",
         algo="HDBSCAN", params={"min_cluster_size": 50}),
]

# ==== 4) Metrics helpers ====
def safe_internal_scores(X, labels):
    uniq = np.unique(labels)
    if len(uniq) <= 1 or len(uniq) >= len(labels) - 1:
        return dict(silhouette=np.nan, davies_bouldin=np.nan, calinski_harabasz=np.nan)
    return dict(
        silhouette = silhouette_score(X, labels),
        davies_bouldin = davies_bouldin_score(X, labels),
        calinski_harabasz = calinski_harabasz_score(X, labels),
    )

def core_only_scores(X, labels):
    mask = labels != -1
    if mask.sum() < 3:
        return dict(silhouette_core=np.nan, davies_bouldin_core=np.nan,
                    calinski_harabasz_core=np.nan, n_labeled=int(mask.sum()))
    core_X, core_labels = X[mask], labels[mask]
    uniq = np.unique(core_labels)
    if len(uniq) <= 1: return dict(silhouette_core=np.nan, davies_bouldin_core=np.nan,
                                   calinski_harabasz_core=np.nan, n_labeled=int(mask.sum()))
    return dict(
        silhouette_core = silhouette_score(core_X, core_labels),
        davies_bouldin_core = davies_bouldin_score(core_X, core_labels),
        calinski_harabasz_core = calinski_harabasz_score(core_X, core_labels),
        n_labeled = int(mask.sum())
    )

def summarize(labels):
    n_total = len(labels)
    noise = int(np.sum(labels == -1))
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    return dict(n_clusters=n_clusters, noise=noise, noise_rate=noise / n_total if n_total else np.nan)

# ==== 5) Run configs ====
rows = []
for cfg in configs:
    X = get_reduced(cfg["embedding"], cfg["reduction"])
    if cfg["algo"] == "DBSCAN":
        labels = DBSCAN(**cfg["params"]).fit_predict(X)
    elif cfg["algo"] == "HDBSCAN":
        labels = hdbscan.HDBSCAN(**cfg["params"]).fit_predict(X)
    else:
        raise ValueError(cfg["algo"])
    
    colname = f"pred_{cfg['name']}"
    mixtral_df[colname] = labels
    
    internals = safe_internal_scores(X, labels)
    core = core_only_scores(X, labels) if cfg["algo"] in ("DBSCAN", "HDBSCAN") else {}
    summary = summarize(labels)
    
    row = dict(method=cfg["name"], embedding=cfg["embedding"], reduction=cfg["reduction"],
               algo=cfg["algo"], params=json.dumps(cfg["params"]),
               **internals, **core, **summary)
    rows.append(row)

results_df = pd.DataFrame(rows)

# Rank by silhouette_core for density-based
results_df["rank_score"] = results_df.apply(
    lambda r: r["silhouette_core"] if r["algo"] in ("DBSCAN","HDBSCAN") else r["silhouette"],
    axis=1
)

results_sorted = results_df.sort_values("rank_score", ascending=False)

# ==== 6) Save outputs ====
os.makedirs("outputs", exist_ok=True)
mixtral_df.to_csv("outputs/test5k_predictions.csv", index=False)
results_sorted.to_csv("outputs/test5k_clustering_metrics.csv", index=False)

print(" Saved predictions and metrics:")
print(" - outputs/test5k_predictions.csv")
print(" - outputs/test5k_clustering_metrics.csv")

display(results_sorted.round(4))


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 157/157 [00:02<00:00, 55.22it/s]
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed

 Saved predictions and metrics:
 - outputs/test5k_predictions.csv
 - outputs/test5k_clustering_metrics.csv


,method,embedding,reduction,algo,params,silhouette,davies_bouldin,calinski_harabasz,silhouette_core,davies_bouldin_core,calinski_harabasz_core,n_labeled,n_clusters,noise,noise_rate,rank_score
0,dbscan_minilm_none_ms20,all-MiniLM-L6-v2,none,DBSCAN,"{""eps"": 0.3, ""min_samples"": 20}",0.0817,0.9786,111.6019,0.8092,0.2518,1476.1123,516,6,4484,0.8968,0.8092
2,dbscan_minilm_none_ms10,all-MiniLM-L6-v2,none,DBSCAN,"{""eps"": 0.3, ""min_samples"": 10}",0.0389,1.0372,63.5030,0.6566,0.4907,791.8459,610,13,4390,0.8780,0.6566
4,hdbscan_minilm_umap_mcs20,all-MiniLM-L6-v2,umap,HDBSCAN,"{""min_cluster_size"": 20}",-0.0056,1.1190,310.9350,0.6465,0.4606,9624.7324,2606,49,2394,0.4788,0.6465
1,dbscan_mpnet_none_ms20,all-mpnet-base-v2,none,DBSCAN,"{""eps"": 0.3, ""min_samples"": 20}",0.0329,1.0786,93.1226,0.6355,0.4553,1107.1167,567,7,4433,0.8866,0.6355
3,hdbscan_paraphrase_umap_mcs10,paraphrase-MiniLM-L3-v2,umap,HDBSCAN,"{""min_cluster_size"": 10}",0.1097,0.9048,488.2528,0.6116,0.4886,8419.0742,3015,83,1985,0.3970,0.6116
5,hdbscan_bge_umap_mcs50,bge-base-en,umap,HDBSCAN,"{""min_cluster_size"": 50}",0.4876,1.2829,1417.2156,0.5105,0.3954,2554.4536,4800,3,200,0.0400,0.5105


In [9]:
mixtral_df

,utterance,Llama3_Code,Falcon_Code,Mistral_Code,Generated_Code,Final_Code,text,pred_dbscan_minilm_none_ms20,pred_dbscan_mpnet_none_ms20,pred_dbscan_minilm_none_ms10,pred_hdbscan_paraphrase_umap_mcs10,pred_hdbscan_minilm_umap_mcs20,pred_hdbscan_bge_umap_mcs50
0,"Prima. Dat geleiden wij door. Mevrouw Bromet, ...","Main idea: ""Mevrouw Bromet, as chair of the Eu...",The main idea of the sentence is that the spea...,"This sentence is about Mrs. Bromet, as the cha...","""Chair of European Affairs, Mevrouw Bromet""","""Chair of European Affairs, Mevrouw Bromet""","""Chair of European Affairs, Mevrouw Bromet"" Pr...",-1,-1,-1,-1,47,2
1,"Voorzitter, dat wilde ik net zeggen. Als voorz...",Main idea: Request for a debate on the informa...,The EU Commission President is requesting a de...,The chairman wishes to propose a debate on the...,"""Debate request for EU leaders' meeting""","""Debate request for EU leaders' meeting""","""Debate request for EU leaders' meeting"" Voorz...",-1,-1,-1,68,22,2
2,Prima. Dat gaan we inplannen. Mevrouw Mutluer ...,"Main idea: ""Planning meeting"" </assistant><|eo...",We will plan to do something.,"The speaker, Mrs. Mutluer from the GroenLinks-P","""Planning meeting with GroenLinks-PvdA""","""Planning meeting with GroenLinks-PvdA""","""Planning meeting with GroenLinks-PvdA"" Prima....",-1,-1,-1,46,-1,2
3,"Dank u wel, voorzitter. Naar aanleiding van ee...","Main idea: ""Parliamentary investigation into i...",The main idea of the sentence is that the spea...,This sentence is about a parliamentary investi...,"""Parliamentary investigation progress""","""Parliamentary investigation progress""","""Parliamentary investigation progress"" Dank u ...",-1,-1,-1,-1,33,2
4,"Dit is wel verwarrend, want in de aankondiging...",Main idea: Request to discuss the current situ...,The main idea of the sentence is that the NRC ...,This sentence is about a confusion regarding a...,Request for clarity on GGZ/police sequence,Request for clarity on GGZ/police sequence,Request for clarity on GGZ/police sequence Dit...,-1,-1,-1,78,-1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Ik wil ook de link met het extremisme- en terr...,Extremism and terrorism debate.<|eot_id|><|sta...,The main idea of the sentence is that the spea...,This sentence is about the speaker wanting to ...,Understanding extremism-debate link,Understanding extremism-debate link,Understanding extremism-debate link Ik wil ook...,-1,-1,-1,68,-1,2
4996,"Waarom nu? Omdat maandag de herdenking, Ketiko...","Main idea: ""The motion is being proposed becau...",The main idea of the sentence is that the spea...,This sentence is about a motion in the Dutch S...,"Motion proposal for unity, regretting divisive...","Motion proposal for unity, regretting divisive...","Motion proposal for unity, regretting divisive...",-1,-1,-1,-1,41,2
4997,90-7-2 27 juni 2024 TK 90 Terrorisme/extremism...,"""Terrorisme/extremisme"" (main idea) </assistant>",The main idea of the sentence is that the Dutc...,This sentence describes a parliamentary sessio...,"""Parliamentary Terrorism-Extremism Debate""","""Parliamentary Terrorism-Extremism Debate""","""Parliamentary Terrorism-Extremism Debate"" 90-...",-1,-1,-1,-1,-1,2
4998,We kunnen een heel debat gaan voeren over wat ...,Main idea: Both parties have made a joint deci...,The main idea of the sentence is that both par...,This sentence is about a debate over a moon-da...,Joint decision on moon-day event,Joint decision on moon-day event,Joint decision on moon-day event We kunnen een...,-1,-1,-1,75,41,2


# Autolabeling

In [20]:
# --- CONFIG ---
METHOD_COLS = [
    "pred_dbscan_minilm_none_ms20",
    "pred_dbscan_mpnet_none_ms20",
    "pred_dbscan_minilm_none_ms10",
    "pred_hdbscan_paraphrase_umap_mcs10",
    "pred_hdbscan_minilm_umap_mcs20",
    "pred_hdbscan_bge_umap_mcs50",
]

GROQ_MODEL = "llama-3.3-70b-versatile"   # good, concise labels
EXAMPLES_PER_CLUSTER = 5
MAX_RETRIES = 3
SLEEP_SEC = 1.5
CACHE_DIR = "cluster_name_cache"          # json files per method here
OUTPUT_DIR = "outputs"                    # csvs written here
CODE_COL = "Final_Code"
UTT_COL = "utterance"

# --- SET YOUR KEY ---
GROQ_API_KEY = ''
USE_GROQ = True   # set False to dry-run / skip API calls


In [24]:
import os, json, time, textwrap
from pathlib import Path
import pandas as pd
from groq import Groq

Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

def _load_cache(method_name: str) -> dict:
    fp = Path(CACHE_DIR) / f"{method_name}.json"
    if fp.exists():
        try:
            return json.loads(fp.read_text(encoding="utf-8"))
        except Exception:
            return {}
    return {}

def _save_cache(method_name: str, cache: dict):
    fp = Path(CACHE_DIR) / f"{method_name}.json"
    fp.write_text(json.dumps(cache, ensure_ascii=False, indent=2), encoding="utf-8")

def _format_examples(df: pd.DataFrame, k: int) -> str:
    ex = df[[CODE_COL, UTT_COL]].dropna().head(k)
    lines = []
    for _, r in ex.iterrows():
        code = str(r[CODE_COL])[:180].replace("\n"," ").strip()
        utt  = str(r[UTT_COL])[:300].replace("\n"," ").strip()
        lines.append(f'Code: "{code}"\nUtterance: "{utt}"')
    return "\n\n".join(lines)

def _prompt_for_examples(n: int, body: str) -> str:
    return textwrap.dedent(f"""
    You are a political discourse analyst performing axial coding.

    Below are {n} coded examples sampled from a single cluster of parliamentary debate utterances:

    {body}

    Task: Propose ONE succinct axial code (≤ 5 words) that best names the cluster.
    Requirements: be specific, topical (not procedural like “discussion”), and avoid punctuation.
    Output ONLY the label.
    """).strip()

def _call_groq(client, prompt: str, model: str, retries: int, sleep_sec: float) -> str:
    last_err = None
    for _ in range(retries):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system",
                     "content": "You are an expert in qualitative research and political discourse analysis. Produce concise, specific axial codes."},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.2,
                max_tokens=16,
            )
            label = (resp.choices[0].message.content or "").strip()
            print(label)
            # sanitize
            label = label.strip('"').strip().replace(":", "").replace(",", "")
            # keep ≤ 5 words
            label = " ".join(label.split()[:5])
            return label if label else "Unlabeled cluster"
        except Exception as e:
            last_err = e
            time.sleep(sleep_sec)
    return "Unlabeled cluster"

def autoname_clusters_with_groq(
        df: pd.DataFrame,
        cluster_col: str,
        api_key: str,
        model: str = GROQ_MODEL,
        examples_per_cluster: int = EXAMPLES_PER_CLUSTER,
        max_retries: int = MAX_RETRIES,
        sleep_sec: float = SLEEP_SEC,
) -> dict:
    """
    Returns {cluster_id: 'Short Label'} mapping for cluster_col.
    Skips -1 (noise) if present.
    """
    cache = _load_cache(cluster_col)
    names = dict(cache)  # start from cache

    # Work subset: exclude noise if exists
    if (df[cluster_col] == -1).any():
        work = df[df[cluster_col] != -1]
    else:
        work = df

    if USE_GROQ:
        client = Groq(api_key=api_key)
    else:
        client = None  # dry-run

    for cid, g in work.groupby(cluster_col, sort=True):
        if str(cid) in names:  # cached
            continue

        if g.empty:
            names[str(cid)] = "Unlabeled cluster"
            continue

        body = _format_examples(g, examples_per_cluster)
        prompt = _prompt_for_examples(min(len(g), examples_per_cluster), body)

        if not USE_GROQ:
            # offline fallback: short TF-IDF-ish heuristic
            # (use most frequent 2 words from codes as a placeholder)
            tokens = " ".join(g[CODE_COL].astype(str).tolist()).lower().split()
            stop = set("the a an and of for to with in on at as is are be this that".split())
            keep = [t.strip('",.;:?!()') for t in tokens if t.isalpha() and t not in stop]
            from collections import Counter
            common = [w for w, _ in Counter(keep).most_common(3)]
            label = " ".join(common[:3]) or "Unlabeled cluster"
        else:
            label = _call_groq(client, prompt, model, max_retries, sleep_sec)

        names[str(cid)] = label
        # persist incrementally
        _save_cache(cluster_col, names)

    return names

def apply_cluster_names(df: pd.DataFrame, cluster_col: str, mapping: dict) -> pd.Series:
    """
    Returns a new pandas Series with cluster names aligned to df rows.
    Noise stays as 'Noise' if present.
    """
    def _map(v):
        if v == -1:
            return "Noise"
        return mapping.get(str(v)) or mapping.get(v) or "Unlabeled cluster"
    return df[cluster_col].map(_map)

def run_autonaming_for_methods(df: pd.DataFrame, methods: list, api_key: str):
    all_maps = {}
    for col in methods:
        print(f"==> Naming clusters for: {col}")
        mapping = autoname_clusters_with_groq(df, col, api_key=api_key)
        all_maps[col] = mapping

        # apply back to df as a new column
        name_col = f"{col}__name"
        df[name_col] = apply_cluster_names(df, col, mapping)

        # save per-method mapping table
        mp = (
            pd.DataFrame(
                [(k, v) for k, v in mapping.items()],
                columns=["cluster_id", "cluster_name"]
            )
            .sort_values("cluster_id", key=lambda s: s.astype(int), ignore_index=True)
        )
        mp.to_csv(os.path.join(OUTPUT_DIR, f"{col}_cluster_names.csv"), index=False)

    # save combined (wide) mapping for convenience
    combined = []
    for method, mp in all_maps.items():
        for cid, nm in mp.items():
            combined.append({"method": method, "cluster_id": cid, "cluster_name": nm})
    pd.DataFrame(combined).to_csv(os.path.join(OUTPUT_DIR, "all_methods_cluster_names.csv"), index=False)

    # and save the annotated dataframe (with __name columns)
    df.to_csv(os.path.join(OUTPUT_DIR, "test5k_with_cluster_names.csv"), index=False)
    return all_maps




In [25]:
# ---- RUN (set USE_GROQ True and provide key) ----
all_maps = run_autonaming_for_methods(mixtral_df, METHOD_COLS, api_key=GROQ_API_KEY)

==> Naming clusters for: pred_dbscan_minilm_none_ms20
Expression of Support
Shows Parliamentary Approval
Acknowledges Appreciation
Van der Lee motions
Vote Outcome Announcements
Opposition to the proposal
==> Naming clusters for: pred_dbscan_mpnet_none_ms20
Expression of Support
Shows Parliamentary Approval
Acknowledges Appreciation
Motion Proposal Attribution
Amendment Rejection Announcement
Vote Outcome Announcements
Lack of Political Support
==> Naming clusters for: pred_dbscan_minilm_none_ms10
Chairman Support Expression
Expression of Support
Shows Parliamentary Approval
Acknowledges Appreciation
Van der Lee motions
Amendment Voting Results
Amendment Voting Outcome
Motion Vote Results
Motion Approval Announcements
Parliament Meeting Introductions
Lack of Political Support
Support for Legislation
Conclusion In Dutch
==> Naming clusters for: pred_hdbscan_paraphrase_umap_mcs10
Conclusion In Dutch
Motion Approval Announcements
Shows Parliamentary Approval
Shows Parliamentary Support
Ac

In [31]:
mixtral_df.to_csv('mixtral_df_with_topics.csv')

In [32]:
mixtral_df

,utterance,Llama3_Code,Falcon_Code,Mistral_Code,Generated_Code,Final_Code,text,pred_dbscan_minilm_none_ms20,pred_dbscan_mpnet_none_ms20,pred_dbscan_minilm_none_ms10,...,pred_hdbscan_bge_umap_mcs50,pred_dbscan_minilm_none_ms20__name,pred_dbscan_mpnet_none_ms20__name,pred_dbscan_minilm_none_ms10__name,pred_hdbscan_paraphrase_umap_mcs10__name,pred_hdbscan_minilm_umap_mcs20__name,pred_hdbscan_bge_umap_mcs50__name,topic,topic_1,topic_2
0,"Prima. Dat geleiden wij door. Mevrouw Bromet, ...","Main idea: ""Mevrouw Bromet, as chair of the Eu...",The main idea of the sentence is that the spea...,"This sentence is about Mrs. Bromet, as the cha...","""Chair of European Affairs, Mevrouw Bromet""","""Chair of European Affairs, Mevrouw Bromet""","""Chair of European Affairs, Mevrouw Bromet"" Pr...",-1,-1,-1,...,2,Noise,Noise,Noise,Noise,Speaker Recognition Moments,EU Policy Initiatives,Bestuur | Parlement,Bestuur,Parlement
1,"Voorzitter, dat wilde ik net zeggen. Als voorz...",Main idea: Request for a debate on the informa...,The EU Commission President is requesting a de...,The chairman wishes to propose a debate on the...,"""Debate request for EU leaders' meeting""","""Debate request for EU leaders' meeting""","""Debate request for EU leaders' meeting"" Voorz...",-1,-1,-1,...,2,Noise,Noise,Noise,EU Leaders Meeting Debate,EU Leaders Meeting Debate,EU Policy Initiatives,Bestuur | Parlement,Bestuur,Parlement
2,Prima. Dat gaan we inplannen. Mevrouw Mutluer ...,"Main idea: ""Planning meeting"" </assistant><|eo...",We will plan to do something.,"The speaker, Mrs. Mutluer from the GroenLinks-P","""Planning meeting with GroenLinks-PvdA""","""Planning meeting with GroenLinks-PvdA""","""Planning meeting with GroenLinks-PvdA"" Prima....",-1,-1,-1,...,2,Noise,Noise,Noise,GroenLinksPvdA Representation,Noise,EU Policy Initiatives,Bestuur | Parlement,Bestuur,Parlement
3,"Dank u wel, voorzitter. Naar aanleiding van ee...","Main idea: ""Parliamentary investigation into i...",The main idea of the sentence is that the spea...,This sentence is about a parliamentary investi...,"""Parliamentary investigation progress""","""Parliamentary investigation progress""","""Parliamentary investigation progress"" Dank u ...",-1,-1,-1,...,2,Noise,Noise,Noise,Noise,Investigation and Minister Response,EU Policy Initiatives,Bestuur | Parlement,Bestuur,Parlement
4,"Dit is wel verwarrend, want in de aankondiging...",Main idea: Request to discuss the current situ...,The main idea of the sentence is that the NRC ...,This sentence is about a confusion regarding a...,Request for clarity on GGZ/police sequence,Request for clarity on GGZ/police sequence,Request for clarity on GGZ/police sequence Dit...,-1,-1,-1,...,2,Noise,Noise,Noise,GGZ Reform Support,Noise,EU Policy Initiatives,Bestuur | Parlement,Bestuur,Parlement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Ik wil ook de link met het extremisme- en terr...,Extremism and terrorism debate.<|eot_id|><|sta...,The main idea of the sentence is that the spea...,This sentence is about the speaker wanting to ...,Understanding extremism-debate link,Understanding extremism-debate link,Understanding extremism-debate link Ik wil ook...,-1,-1,-1,...,2,Noise,Noise,Noise,EU Leaders Meeting Debate,Noise,EU Policy Initiatives,Openbare orde en veiligheid | Terrorisme,Openbare orde en veiligheid,Terrorisme
4996,"Waarom nu? Omdat maandag de herdenking, Ketiko...","Main idea: ""The motion is being proposed becau...",The main idea of the sentence is that the spea...,This sentence is about a motion in the Dutch S...,"Motion proposal for unity, regretting divisive...","Motion proposal for unity, regretting divisive...","Motion proposal for unity, regretting divisive...",-1,-1,-1,...,2,Noise,Noise,Noise,Noise,Headlines Agreement Negotiation,EU Policy Initiatives,Openbare orde en veiligheid | Terrorisme,Openbare orde en veiligheid,Terrorisme
4997,90-7-2 27 juni 2024 TK 90 Terrorisme/extremism...,"""Terrorisme

In [43]:
import pandas as pd
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import evaluate  # huggingface evaluate library

# === Load data ===
df = pd.read_csv("mixtral_df_with_topics_en.csv")

# cluster label columns (replace with yours if different)
CLUSTER_COLS = [
    "pred_dbscan_minilm_none_ms10__name",
    "pred_dbscan_minilm_none_ms20__name",
    "pred_dbscan_mpnet_none_ms20__name",
    "pred_hdbscan_paraphrase_umap_mcs10__name",
    "pred_hdbscan_minilm_umap_mcs20__name",
    "pred_hdbscan_bge_umap_mcs50__name",
]

TOPIC_COLS = ["topic_1", "topic_2"]

# === Setup scorers ===
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
model = SentenceTransformer("all-MiniLM-L6-v2")
bertscore = evaluate.load("bertscore")

def compute_scores(preds, refs):
    """Compute ROUGE (1/2/L), cosine sim, and BERTScore (F1)."""
    # ROUGE
    r1, r2, rL = [], [], []
    for p, r in zip(preds, refs):
        scores = rouge.score(r, p)
        r1.append(scores['rouge1'].fmeasure)
        r2.append(scores['rouge2'].fmeasure)
        rL.append(scores['rougeL'].fmeasure)
    rouge1_avg = sum(r1) / len(r1)
    rouge2_avg = sum(r2) / len(r2)
    rougeL_avg = sum(rL) / len(rL)

    # Cosine similarity
    emb_pred = model.encode(preds, convert_to_tensor=True, show_progress_bar=False)
    emb_ref  = model.encode(refs, convert_to_tensor=True, show_progress_bar=False)
    cos_sims = util.cos_sim(emb_pred, emb_ref).diagonal().cpu().tolist()
    cos_avg  = sum(cos_sims) / len(cos_sims)

    # BERTScore (F1)
    bert = bertscore.compute(predictions=preds, references=refs, lang="en")
    bert_f1 = sum(bert["f1"]) / len(bert["f1"])

    return rouge1_avg, rouge2_avg, rougeL_avg, cos_avg, bert_f1

# === Run evaluation ===
results = []
for col in CLUSTER_COLS:
    for tcol in TOPIC_COLS:
        work = df[(df[col].notna()) & (df[col] != "Noise") & df[tcol].notna()]
        if work.empty:
            continue
        r1, r2, rL, cos_avg, bert_f1 = compute_scores(
            work[col].astype(str).tolist(),
            work[tcol].astype(str).tolist()
        )
        results.append({
            "method": col,
            "topic_ref": tcol,
            "rouge1_f1": r1,
            "rouge2_f1": r2,
            "rougeL_f1": rL,
            "cosine_sim": cos_avg,
            "bertscore_f1": bert_f1,
            "n": len(work)
        })

results_df = pd.DataFrame(results)
print(results_df)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                      method topic_ref  rouge1_f1  rouge2_f1  \
0         pred_dbscan_minilm_none_ms10__name   topic_1   0.000000   0.000000   
1         pred_dbscan_minilm_none_ms10__name   topic_2   0.008197   0.000000   
2         pred_dbscan_minilm_none_ms20__name   topic_1   0.000000   0.000000   
3         pred_dbscan_minilm_none_ms20__name   topic_2   0.000000   0.000000   
4          pred_dbscan_mpnet_none_ms20__name   topic_1   0.000000   0.000000   
5          pred_dbscan_mpnet_none_ms20__name   topic_2   0.000000   0.000000   
6   pred_hdbscan_paraphrase_umap_mcs10__name   topic_1   0.004407   0.000000   
7   pred_hdbscan_paraphrase_umap_mcs10__name   topic_2   0.027466   0.000398   
8       pred_hdbscan_minilm_umap_mcs20__name   topic_1   0.014531   0.000000   
9       pred_hdbscan_minilm_umap_mcs20__name   topic_2   0.037493   0.000000   
10         pred_hdbscan_bge_umap_mcs50__name   topic_1   0.000000   0.000000   
11         pred_hdbscan_bge_umap_mcs50__

In [44]:
results_df

,method,topic_ref,rouge1_f1,rouge2_f1,rougeL_f1,cosine_sim,bertscore_f1,n
0,pred_dbscan_minilm_none_ms10__name,topic_1,0.000000,0.000000,0.000000,0.098430,0.831564,610
1,pred_dbscan_minilm_none_ms10__name,topic_2,0.008197,0.000000,0.008197,0.121151,0.835137,610
2,pred_dbscan_minilm_none_ms20__name,topic_1,0.000000,0.000000,0.000000,0.107279,0.828365,516
3,pred_dbscan_minilm_none_ms20__name,topic_2,0.000000,0.000000,0.000000,0.142583,0.831552,516
4,pred_dbscan_mpnet_none_ms20__name,topic_1,0.000000,0.000000,0.000000,0.112810,0.839660,567
5,pred_dbscan_mpnet_none_ms20__name,topic_2,0.000000,0.000000,0.000000,0.148045,0.844268,567
6,pred_hdbscan_paraphrase_umap_mcs10__name,topic_1,0.004407,0.000000,0.004407,0.125892,0.835243,3015
7,pred_hdbscan_paraphrase_umap_mcs10__name,topic_2,0.027466,0.000398,0.027466,0.168463,0.842049,3015
8,pred_hdbscan_minilm_umap_mcs20__name,topic_1,0.014531,0.000000,0.014531,0.138114,0.835147,2606
9,pred_hdbscan_minilm_umap_mcs20__name,topic_2,0.037493,0.000000,0.037493,0.183096,0.842386,2606
